In [2]:
import numpy as np
import time
from tqdm import tqdm
import re
import os

# 定义调制类型的顺序
modulation_order = [
    'OOK', '4ASK', '8ASK', 'BPSK', 'QPSK', '8PSK', '16PSK', '32PSK',
    '16APSK', '32APSK', '64APSK', '128APSK', '16QAM', '32QAM', '64QAM',
    '128QAM', '256QAM', 'AM-SSB-WC', 'AM-SSB-SC', 'AM-DSB-WC', 'AM-DSB-SC',
    'FM', 'GMSK', 'OQPSK'
]

# 查找文件夹中的所有文件
file_dir = './selected_data/SNR_6/'  # 假设所有文件都在当前文件夹中
files = os.listdir(file_dir)

# 使用正则表达式查找文件名中的 SNR 值
snr_pattern = re.compile(r"X_SNR_(\d+)_mod_(.*).npy")

# 创建空列表来存储所有的numpy数组
data_list = []
snr_value = None

# 显示总开始时间
start_time = time.time()

# 循环读取并堆叠数据，使用 tqdm 显示进度条
for mod in tqdm(modulation_order, desc="Processing modulation files"):
    # 根据调制类型找到匹配的文件
    matching_files = [f for f in files if f"mod_{mod}" in f and snr_pattern.search(f)]
    
    if matching_files:
        # 处理第一个匹配的文件（假设每种调制方式只有一个 SNR 对应）
        file_name = os.path.join(file_dir, matching_files[0])
        
        # 提取 SNR 值
        match = snr_pattern.search(file_name)
        if match:
            snr_value = match.group(1)  # 提取 SNR 的数值
            mod_type = match.group(2)   # 提取调制类型
            
            # 记录每个文件处理的开始时间
            file_start_time = time.time()
            
            # 读取npy文件
            data = np.load(file_name)
            
            # 将每个数据存入列表
            data_list.append(data)
            
            # 记录文件处理结束时间并计算耗时
            file_end_time = time.time()
            print(f"Processed {file_name} (SNR={snr_value}) in {file_end_time - file_start_time:.2f} seconds")
    else:
        print(f"No file found for modulation {mod}")

# 确保SNR值被正确识别
if snr_value is None:
    raise ValueError("No SNR value found. Please check the file naming format.")

# 将列表堆叠成形状为 (24, 1000, 1024, 2) 的数组
combined_data = np.stack(data_list, axis=0)
cls, smaples, single, iq = combined_data.shape
print(f"combined_data.shape:{combined_data.shape}")

# 保存到新的npy文件，文件名包含动态的 SNR 值
output_file_name = f'stacked_modulation_data_SNR_{snr_value}_k{smaples}.npy'
output_file_name = os.path.join(file_dir, output_file_name)
np.save(output_file_name, combined_data)

# 显示总耗时
end_time = time.time()
print(f"All files processed and saved in {end_time - start_time:.2f} seconds. Saved as {output_file_name}.")


FileNotFoundError: [Errno 2] No such file or directory: './selected_data/SNR_6/'